In [210]:
import pandas as pd
import torch
import PIL.Image
from torchvision import transforms
import  numpy as np
import  torch
import torchvision
import torch.nn.functional as F
from torch import  nn
from torch import optim

In [211]:

data_file = "data/lungdatasets.csv"

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('max_colwidth', 100)

In [212]:
# csv使用分号分割，seq指定分隔符为；
data = pd.read_csv(data_file,sep=';')

# 住院时长周数为值
outputs = data.iloc[:, 25]

# 定义特征
all_features = data.iloc[:, 1:25]
strlist = ['性别(1=男，2=女）','入院时临床分型（0=无症状感染者，1=轻型，2=普通型，3=重型，4=危重型）','出院时临床分型（0=无症状感染者，1=轻型，2=普通型，3=重型，4=危重型）','出院时临床转归（0=临床治愈，1=病毒转阴）','首诊时有无发热（0=否，1=是）','以前曾是否患过肺炎（0=否，1=是）','病灶位置是否右上（1=是，0=否）','病灶位置是否右中（1=是，0=否）','病灶位置是否右下（1=是，0=否）','病灶位置是否左上（1=是，0=否）','病灶位置是否左下（1=是，0=否）']
all_features[strlist] = all_features[strlist].astype(np.str)
print(all_features.dtypes)


年龄(岁)                                      float64
性别(1=男，2=女）                                 object
入院时临床分型（0=无症状感染者，1=轻型，2=普通型，3=重型，4=危重型）     object
出院时临床分型（0=无症状感染者，1=轻型，2=普通型，3=重型，4=危重型）     object
出院时临床转归（0=临床治愈，1=病毒转阴）                      object
首次查体体温（℃）                                  float64
首诊时有无发热（0=否，1=是）                            object
身高（cm）                                     float64
体重(kg)                                     float64
既往史评分                                        int64
以前曾是否患过肺炎（0=否，1=是）                          object
ICU时长（天）                                     int64
呼吸机时长（小时）                                  float64
病灶位置是否右上（1=是，0=否）                           object
病灶位置是否右中（1=是，0=否）                           object
病灶位置是否右下（1=是，0=否）                           object
病灶位置是否左上（1=是，0=否）                           object
病灶位置是否左下（1=是，0=否）                           object
首次检查病灶体积总和（cm?）                            float64
首次检查全部病灶占全肺体积占比(%)             

In [213]:
# 若无法获得测试数据，则可根据训练数据计算均值和标准差
numeric_features = all_features.dtypes[all_features.dtypes != 'object'].index

# print(numeric_features[1])
# print(all_features[numeric_features])

all_features[numeric_features] = all_features[numeric_features].apply(
    lambda x: (x - x.mean()) / (x.std()))
# 在标准化数据之后，所有均值消失，因此我们可以将缺失值设置为0
all_features[numeric_features] = all_features[numeric_features].fillna(0)

# “Dummy_na=True”将“na”（缺失值）视为有效的特征值，并为其创建指示符特征
all_features = pd.get_dummies(all_features, dummy_na=False)

all_features

,年龄(岁),首次查体体温（℃）,身高（cm）,体重(kg),既往史评分,ICU时长（天）,呼吸机时长（小时）,首次检查病灶体积总和（cm?）,首次检查全部病灶占全肺体积占比(%),首次检查全部病灶的平均密度（HU）,末次检查病灶体积总和（cm?）,末次检查全部病灶占全肺体积占比(%),末次检查全部病灶的平均密度（HU）,性别(1=男，2=女）_1,性别(1=男，2=女）_2,入院时临床分型（0=无症状感染者，1=轻型，2=普通型，3=重型，4=危重型）_0.0,入院时临床分型（0=无症状感染者，1=轻型，2=普通型，3=重型，4=危重型）_1.0,入院时临床分型（0=无症状感染者，1=轻型，2=普通型，3=重型，4=危重型）_2.0,入院时临床分型（0=无症状感染者，1=轻型，2=普通型，3=重型，4=危重型）_3.0,入院时临床分型（0=无症状感染者，1=轻型，2=普通型，3=重型，4=危重型）_4.0,入院时临床分型（0=无症状感染者，1=轻型，2=普通型，3=重型，4=危重型）_nan,出院时临床分型（0=无症状感染者，1=轻型，2=普通型，3=重型，4=危重型）_0.0,出院时临床分型（0=无症状感染者，1=轻型，2=普通型，3=重型，4=危重型）_1.0,出院时临床分型（0=无症状感染者，1=轻型，2=普通型，3=重型，4=危重型）_2.0,出院时临床分型（0=无症状感染者，1=轻型，2=普通型，3=重型，4=危重型）_3.0,出院时临床分型（0=无症状感染者，1=轻型，2=普通型，3=重型，4=危重型）_4.0,出院时临床分型（0=无症状感染者，1=轻型，2=普通型，3=重型，4=危重型）_nan,出院时临床转归（0=临床治愈，1=病毒转阴）_0.0,出院时临床转归（0=临床治愈，1=病毒转阴）_1.0,出院时临床转归（0=临床治愈，1=病毒转阴）_nan,首诊时有无发热（0=否，1=是）_0.0,首诊时有无发热（0=否，1=是）_1.0,首诊时有无发热（0=否，1=是）_nan,以前曾是否患过肺炎（0=否，1=是）_0.0,以前曾是否患过肺炎（0=否，1=是）_1.0,以前曾是否患过肺炎（0=否，1=是）_nan,病灶位置是否右上（1=是，0=否）_0,病灶位置是否右上（1=是，0=否）_1,病灶位置是否右中（1=是，0=否）_0,病灶位置是否右中（1=是，0=否）_1,病灶位置是否右下（1=是，0=否）_0,病灶位置是否右下（1=是，0=否）_1,病灶位置是否左上（1=是，0=否）_0,病灶位置是否左上（1=是，0=否）_1,病灶位置是否左下（1=是，0=否）_0.0,病灶位置是否左下（1=是，0=否）_1.0,病灶位置是否左下（1=是，0=否）_nan
0,-0.942885,1.080185,0.506741,0.358882,2.316212,-0.196760,-0.145085,0.139141,0.024206,0.012886,-0.264087,-0.187577,0.781205,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,1,0,0,1,0,1,0,0,1,0,1,0,1,0
1,-0.690293,-0.915005,0.277314,0.271939,-0.423394,-0.196760,-0.145085,-0.368587,-0.082563,-1.054616,-0.264087,-0.187577,0.781205,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,1,0,0,1,0,1,0,0,1,1,0,1,0,0
2,-0.627145,-1.501826,0.485884,0.163261,-0.423394,-0.196760,-0.145085,-0.460993,-0.100527,1.263281,-0.264087,-0.187577,0.781205,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,1,0,0,1,0,1,0,1,0,1,0,1,0,0
3,0.067482,-0.915005,0.089601,0.380617,-0.423394,-0.196760,-0.145085,-0.460993,-0.100527,1.263281,-0.264087,-0.187577,0.781205,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,1,0,0,1,0,1,0,1,0,1,0,1,0,0
4,-0.690293,2.019098,0.173029,0.271939,-0.423394,-0.196760,-0.145085,-0.306250,-0.059854,-0.815734,-0.264087,-0.187577,0.781205,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,1,0,0,1,0,1,0,0,1,0,1,0,1,0
5,-1.132328,-1.501826,0.214743,-0.727901,0.946409,-0.196760,-0.145085,-0.460993,-0.100527,1.263281,-0.231495,-0.176443,-1.061305,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,1,0,0,1,0,1,0,1,0,1,0,1,0,0
6,1.330441,0.610728,0.277314,0.119789,0.946409,1.926322,1.818024,1.211527,0.753284,-0.028172,-0.264087,-0.187577,0.781205,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,1,0,1,0,0,0,1,0,1,0,1,0,1,0,1,0
7,1.393589,0.376000,0.506741,-0.075831,0.946409,2.398118,4.921486,2.358947,0.655328,-0.446215,-0.246506,-0.178918,-1.404824,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,1,0,0,0,1,0,1,0,1,0,1,0,1,0
8,1.393589,0.610728,0.173029,0.402353,0.946409,-0.196760,-0.145085,0.392081,0.089623,-1.185254,-0.216957,-0.167784,-1.610936,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,1,0,0,0,1,0,1,0,1,0,1,0,1,0
9,0.509518,0.258636,0.000000,0.000000,-0.423394,-0.196760,-0.145085,-0.460993,-0.100527,1.263281,0.000000,0.000000,0.000000,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,1,0,0,1,0,1,0,1,0,1,0,1,0,0


In [214]:
#输入
data = torch.tensor(all_features.values)
#输出
target = torch.tensor(outputs.values)

data_dim = data.shape[1]
# target_dim = target.shape[1]
print(data)
print(data.shape)
print(data_dim,target.shape)
target.unique()

tensor([[-0.9429,  1.0802,  0.5067,  ...,  0.0000,  1.0000,  0.0000],
        [-0.6903, -0.9150,  0.2773,  ...,  1.0000,  0.0000,  0.0000],
        [-0.6271, -1.5018,  0.4859,  ...,  1.0000,  0.0000,  0.0000],
        ...,
        [ 0.3201, -0.7976,  0.3816,  ...,  0.0000,  1.0000,  0.0000],
        [-0.8797, -0.5629,  0.1730,  ...,  1.0000,  0.0000,  0.0000],
        [ 0.5095, -0.6803,  0.0000,  ...,  1.0000,  0.0000,  0.0000]],
       dtype=torch.float64)
torch.Size([440, 47])
47 torch.Size([440])


tensor([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])

In [215]:
from torch.utils.data import Dataset


class LungDataset(Dataset):

    def __init__(self,inputs,outputs):
        self.inputs = inputs
        self.outputs = outputs

    def __getitem__(self, idx):
        vector = self.inputs[idx]
        label = self.outputs[idx]
        return vector,label

    def __len__(self):
        return len(self.outputs)

lungdataset = LungDataset(data,outputs)
x,y = lungdataset[0]
x.shape



torch.Size([47])

In [216]:
#搭建网络
class MnistNet(nn.Module):
    def __init__(self):
        super(MnistNet,self).__init__()
        self.fc1 = nn.Linear(47*1,28)  #定义Linear的输入和输出的形状
        self.fc2 = nn.Linear(28,12)  #定义Linear的输入和输出的形状

    def forward(self,x):
        x = x.view(-1,47*1)  #对数据形状变形，-1表示该位置根据后面的形状自动调整
        x = self.fc1(x) #[batch_size,28]
        x = F.relu(x)  #[batch_size,28]
        x = self.fc2(x) #[batch_size,10]
        return F.log_softmax(x, dim=-1)

In [217]:
#准备迭代器
train_dataloader = torch.utils.data.DataLoader(data,batch_size=64,shuffle=True)

In [218]:
train_batch_size = 128
test_batch_size = 256
img_size = data_dim

def get_dataloader(train=True):

    #准备数据集，其中0.1307，0.3081为MNIST数据的均值和标准差，这样操作能够对其进行标准化
    #因为MNIST只有一个通道（黑白图片）,所以元组中只有一个值
    dataset = lungdataset
    #准备数据迭代器
    batch_size = train_batch_size if train else test_batch_size
    dataloader = torch.utils.data.DataLoader(dataset,batch_size=batch_size,shuffle=True)
    return dataloader

In [219]:
mnist_net = MnistNet().double()
optimizer = optim.Adam(mnist_net.parameters(),lr= 0.001)
train_loss_list = []
train_count_list = []
def train(epoch):
    mnist_net.train(True)
    train_dataloader = get_dataloader(True)
    print("开始训练：")
    for idx,(data,target) in enumerate(train_dataloader):
        optimizer.zero_grad()
        output = mnist_net(data)
        loss = F.nll_loss(output,target) #对数似然损失
        loss.backward()
        optimizer.step()
        if idx % 200 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(epoch,  idx * len(data), len(train_dataloader.dataset),100. * idx / len(train_dataloader), loss.item()))
            train_loss_list.append(loss.item())
            train_count_list.append(idx*train_batch_size+(epoch-1)*len(train_dataloader))
    print("结束训练。")

In [220]:
epoch = 2000
for i in range(epoch):
    train(i)

开始训练：
Train Epoch: 0 [0/440 (0%)]	Loss: 2.492303
结束训练。
开始训练：
Train Epoch: 1 [0/440 (0%)]	Loss: 2.452814
结束训练。
开始训练：
Train Epoch: 2 [0/440 (0%)]	Loss: 2.433801
结束训练。
开始训练：
Train Epoch: 3 [0/440 (0%)]	Loss: 2.411662
结束训练。
开始训练：
Train Epoch: 4 [0/440 (0%)]	Loss: 2.356211
结束训练。
开始训练：
Train Epoch: 5 [0/440 (0%)]	Loss: 2.313530
结束训练。
开始训练：
Train Epoch: 6 [0/440 (0%)]	Loss: 2.262899
结束训练。
开始训练：
Train Epoch: 7 [0/440 (0%)]	Loss: 2.256599
结束训练。
开始训练：
Train Epoch: 8 [0/440 (0%)]	Loss: 2.196717
结束训练。
开始训练：
Train Epoch: 9 [0/440 (0%)]	Loss: 2.169233
结束训练。
开始训练：
Train Epoch: 10 [0/440 (0%)]	Loss: 2.110827
结束训练。
开始训练：
Train Epoch: 11 [0/440 (0%)]	Loss: 1.995567
结束训练。
开始训练：
Train Epoch: 12 [0/440 (0%)]	Loss: 2.030532
结束训练。
开始训练：
Train Epoch: 13 [0/440 (0%)]	Loss: 1.966795
结束训练。
开始训练：
Train Epoch: 14 [0/440 (0%)]	Loss: 1.953511
结束训练。
开始训练：
Train Epoch: 15 [0/440 (0%)]	Loss: 1.896463
结束训练。
开始训练：
Train Epoch: 16 [0/440 (0%)]	Loss: 1.920654
结束训练。
开始训练：
Train Epoch: 17 [0/440 (0%)]	Loss: 1.894865
结束训练。
开始

In [221]:
def test():
    test_loss = 0
    correct = 0
    mnist_net.eval()
    test_dataloader = get_dataloader(train=False)
    with torch.no_grad():
        for data, target in test_dataloader:
            output = mnist_net(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()
            pred = output.data.max(1, keepdim=True)[1] #获取最大值的位置,[batch_size,1]
            correct += pred.eq(target.data.view_as(pred)).sum()
    test_loss /= len(test_dataloader.dataset)
    print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_dataloader.dataset),
        100. * correct / len(test_dataloader.dataset)))

In [222]:
test()


Test set: Avg. loss: 0.6004, Accuracy: 343/440 (77.95%)



In [223]:
torch.save(mnist_net.state_dict(),"model/mnist_net.pt") #保存模型参数
torch.save(optimizer.state_dict(), 'results/mnist_optimizer.pt') #保存优化器参数

FileNotFoundError: [Errno 2] No such file or directory: 'model/mnist_net.pt'

In [ ]:
mnist_net.load_state_dict(torch.load("model/mnist_net.pt"))
optimizer.load_state_dict(torch.load("results/mnist_optimizer.pt"))